# Practise Pandas and clean merge data

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("/Users/ninawiedemann/Desktop/UNI/Praktikum/merge_16.csv")
# print(file.columns.tolist())
# print(file["Extension (P)"][4])
# file.dtypes()
# print(file.head())
# für array nur von den Daten: file.values()
# file.sort_index(axis = 1, ascending = True) //um überschriften zu sortieren zb nach alphabet
#sorted = file.sort_values(by = "Extension (P)")
# print(a["Extension (P)"])
# für statistics also mean std usw: file.describe()
# print(file[0:3])
# mehrere spalten auswählen: file.loc[:, ['A','B']]


# a= df['Extension (P)'].values
# print(a)
# print(np.any(np.isnan(a)))

df = df.sort_values(by = "Backspin Rate (P)")
print(df["Backspin Rate (P)"])
for i in range(2289,2269, -1):
    df = df.drop(df.index[[i]])
print(df["Backspin Rate (P)"])

In [ ]:
koord = df.iloc[:,df.columns.get_loc("0"):df.columns.get_loc("159")]

"""clean data by deleting nan columns"""
for col in df.columns.tolist():
    if (type(df[col][0]) is not np.float64 and type(df[col][0]) is not np.int64) or np.any(np.isnan(df[col].values)):
        df.drop(col, axis=1, inplace=True)

for col in koord.columns.tolist():
    df[col]=koord[col]

print(df.columns.tolist())

In [ ]:
df.to_csv("/Users/ninawiedemann/Desktop/UNI/Praktikum/merge_clean.csv")

In [ ]:
cl = pd.read_csv("/Users/ninawiedemann/Desktop/UNI/Praktikum/merge_clean.csv")
#print(cl.columns.tolist())
#print(cl["0"])


In [ ]:
cf = pd.read_csv("/Users/ninawiedemann/Desktop/UNI/Praktikum/cf_data.csv")
sv = pd.read_csv("/Users/ninawiedemann/Desktop/UNI/Praktikum/sv_data.csv")

# Get coordinate values and pitch type labels seperately

In [ ]:
coordinates = cf.iloc[:,cf.columns.get_loc("0"):cf.columns.get_loc("166")]

In [ ]:
unique  = np.unique(cf["Pitch Type"].values)

In [ ]:
def oneHot(dataframe, column):
    unique  = np.unique(dataframe[column].values)
    l = len(dataframe.index)
    loc = dataframe.columns.get_loc(column)
    labels = np.zeros((l, 12))
    for i in range(l):
        #print(cf.iloc[i,loc])
        pitch = dataframe.iloc[i,loc]
        ind = unique.tolist().index(pitch)
        labels[i, ind] = 1
    return labels, unique

labels, _ = oneHot(cf, "Pitch Type")

In [ ]:
data_array = coordinates.values

# Change pandas dataframe to np.array
(Problem: missing values in den Koordinaten)

In [ ]:
M, N = data_array.shape
data = np.zeros((M,N,18,2))

for i in range(M):
    for j in range(N):
        if not pd.isnull(data_array[i,j]):
            data[i,j]=np.array(eval(data_array[i,j]))

#print(data)

In [ ]:
missing = pd.isnull(data_array)
print("Ratio missing: ", np.count_nonzero(missing)/(M*N))

# Tensorflow conv net

In [ ]:
train_x = data[:13000,:]
test_x = data[13000:, :]
train_t= labels[:13000,:]
test_t = labels[13000:, :]

print(len(train_x))
print(len(test_x))

In [ ]:
import tensorflow as tf

x = tf.placeholder(tf.float32, (None, N, 18, 2), name = "input")
x_ = tf.reshape(x, (-1, N, 36 ,1))
y = tf.placeholder(tf.float32, (None, len(labels[0])))

net = tf.layers.conv2d(x_, filters=16, kernel_size=5, strides=2, activation=tf.nn.relu)
net = tf.layers.conv2d(net, filters=16, kernel_size=3, strides=1, activation=tf.nn.relu)
net = tf.layers.conv2d(net, filters=32, kernel_size=3, strides=1, activation=tf.nn.relu)
net = tf.layers.conv2d(net, filters=1, kernel_size=1)
shapes = net.get_shape().as_list()
ff = tf.reshape(net, (-1, shapes[1]*shapes[2]))
ff = tf.layers.dense(ff, 128, activation = tf.nn.relu)
ff = tf.layers.dense(ff, len(labels[0]), activation = tf.nn.sigmoid)

loss = tf.reduce_mean(tf.square(y-ff))
optimizer = tf.train.AdamOptimizer(0.0001).minimize(loss)

In [ ]:
def batches(x, y, batchsize=32):
    permute = np.random.permutation(len(x))
    for i in range(0, len(x)-batchsize, batchsize):
        indices = permute[i:i+batchsize]
        yield x[indices], y[indices]
    
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Run session for 2000 epochs
for epoch in range(2000 + 1):
    for batch_x, batch_t in batches(train_x, train_t, 32):
        sess.run(optimizer, {x: batch_x, y: batch_t})
    print(epoch, sess.run(loss, {x: test_x, y: test_t}))
    

In [ ]:
print(sess.run(loss, {x: test_x, y: test_t}))

# one hot encoding

In [ ]:
def decode_one_hot(results):
    """takes the maximum value and gets the corresponding pitch type
    input: array of size trials * pitchTypesNr
    returns: array of size trials containing the pitch type as a string
    """
    unique  = np.unique(cf["Pitch Type"].values)
    p = []
    for pitch, i in enumerate(results):
        ind = np.argmax(pitch)
        p.append(unique[ind])
    return p

testing = sess.run(ff, {x: test_x, y: test_t})
pitches = decode_one_hot(testing)
label_pitches = (cf["Pitch Type"].values)[13000:]

# evaluate error rate
right = 0
wrong = 0
for i in range(len(pitches)):
    if pitches[i]==label_pitches[i]:
        right+=1
    else:
        wrong+=1
print(right/(right+wrong))

In [ ]:
def oneHot(dataframe, column):
    unique  = np.unique(dataframe[column].values)
    l = len(dataframe.index)
    loc = dataframe.columns.get_loc(column)
    labels = np.zeros((l, 12))
    for i in range(l):
        #print(cf.iloc[i,loc])
        pitch = dataframe.iloc[i,loc]
        ind = unique.tolist().index(pitch)
        labels[i, ind] = 1
    return labels, unique

def decode_one_hot(results, unique):
    """takes the maximum value and gets the corresponding pitch type
    input: array of size trials * pitchTypesNr
    returns: array of size trials containing the pitch type as a string
    """
    #unique  = np.unique(cf["Pitch Type"].values)
    p = []
    print(results)
    for i, pitch in enumerate(results):
        #print(pitch)
        ind = np.argmax(pitch)
        #print(ind)
        p.append(unique[ind])
    return p

labels, unique = oneHot(frame, "Pitch Type")

new = decode_one_hot(labels, unique)
print(np.sum(new==frame["Pitch Type"]))

# Only pitcher 

In [ ]:
df = pd.read_csv("/Users/ninawiedemann/Desktop/UNI/Praktikum/cf_data.csv")
#df = df.sort_values(by = "Player") # Ball Picther or Strike outcome pitcher
# bis 8615 ball/pitcher
pitcher = df.loc[frame["Player"]=="Pitcher"]
pitcher.to_csv("/Users/ninawiedemann/Desktop/UNI/Praktikum/cf_only_pitcher.csv")

In [ ]:
frame = pd.read_csv("/Users/ninawiedemann/Desktop/UNI/Praktikum/cf_only_pitcher.csv")
print(len(frame["Player"]))

In [ ]:
a = frame.loc[frame["Player"]=="Pitcher"]

In [ ]:
print(a.iloc[650, a.columns.get_loc("Player")])

In [ ]:
arr1 = (["hi", "ni", "gu"])
arr2 = (["hi", "nina", "gu"])
arr3 = np.asarray(arr1)==np.asarray(arr2)
print(arr3)
print(np.sum(arr3))

print(type(frame["Player"].values[6000:]))

In [ ]:
cf = pd.read_csv("/Users/ninawiedemann/Desktop/UNI/Praktikum/cf_only_pitcher.csv")

print("csv eingelesen")
# COORDINATES
coordinates = cf.iloc[:,cf.columns.get_loc("0"):cf.columns.get_loc("166")]
data_array = coordinates.values
M, N = data_array.shape
data = np.zeros((M,N,18,2))

c=0
for i in range(M):
    for j in range(N):
        if not pd.isnull(data_array[i,j]):
            data[i,j]=np.array(eval(data_array[i,j]))
        else:
            c+=1

np.save("coord_array", data)

In [ ]:
SEP = 6000
EPOCHS = 30

print("imports")
cf = pd.read_csv("/Users/ninawiedemann/Desktop/UNI/Praktikum/cf_only_pitcher.csv")

print("csv eingelesen")
# COORDINATES
"""
coordinates = cf.iloc[:,cf.columns.get_loc("0"):cf.columns.get_loc("166")]
data_array = coordinates.values
M, N = data_array.shape
data = np.zeros((M,N,18,2))

c=0
for i in range(M):
    for j in range(N):
        if not pd.isnull(data_array[i,j]):
            data[i,j]=np.array(eval(data_array[i,j]))
        else:
            c+=1
print("nan in daten:",c)
"""

data = np.load("coord_array.npy")
M,N, _,_ = data.shape

# LABELS
def oneHot(dataframe, column):
    unique  = np.unique(dataframe[column].values)
    l = len(dataframe.index)
    loc = dataframe.columns.get_loc(column)
    labels = np.zeros((l, 12))
    for i in range(l):
        #print(cf.iloc[i,loc])
        pitch = dataframe.iloc[i,loc]
        ind = unique.tolist().index(pitch)
        labels[i, ind] = 1
    return labels, unique

def decode_one_hot(results, unique):
    """takes the maximum value and gets the corresponding pitch type
    input: array of size trials * pitchTypesNr
    returns: array of size trials containing the pitch type as a string
    """
    #unique  = np.unique(cf["Pitch Type"].values)
    p = []
    for _, pitch in enumerate(results):
        ind = np.argmax(pitch)
        p.append(unique[ind])
    return p

labels, unique = oneHot(cf, "Pitch Type")
label_pitches = (cf["Pitch Type"].values)[SEP:]

# NET

np.save("labels", labels)
np.save("unique", unique)
np.save("label_pitches", label_pitches)


In [ ]:
df = pd.read_csv("/Users/ninawiedemann/Desktop/UNI/Praktikum/merge_16.csv")
print(df.columns.tolist())

In [ ]:
df = df.loc[df["Player"] == "Pitcher"]
coordina = df.iloc[:, df.columns.get_loc("0"):df.columns.get_loc("159")]

In [ ]:
M, N = coordina.values.shape
SEP = int(M*0.9)
print("The training data will be up to %d and the test data from %d to %d"%(SEP, SEP, M))

In [ ]:
pointer = df.columns.get_loc("0")
columns = df.columns.tolist()
start = pointer
while(True):
    try:
        zahl = int(columns[pointer])
        pointer+=1
    except ValueError:
        break

coord = df.iloc[:, start:pointer]

# Standardizing and balancing

In [ ]:
import pandas as pd
import numpy as np
cf = pd.read_csv("merge_16.csv")
cf = cf[cf["Player"]=="Pitcher"]

In [ ]:
import scipy as sp
import scipy.stats

print(np.unique(cf["Pitch Type"].values))

types = cf["Pitch Type"].values
note_frequency = sp.stats.itemfreq(types)
print(note_frequency)
smaller_20 = (note_frequency[np.where(note_frequency[:,1]<30)])[:,0].flatten()

for typ in smaller_20:
    cf = cf.drop(cf[cf["Pitch Type"]==typ].index)


print(np.unique(cf["Pitch Type"].values))
#cf = cf.drop(cf[cf["Pitch Type"]=='Fastball (Split-finger)' or cf["Pitch Type"]=="Unknown Pitch Type"].index)

In [ ]:
arr = np.load("coord_array.npy")
print(np.any(np.isnan(arr)))
means = np.mean(arr, axis = 1)
std = np.std(arr, axis = 1)
res = np.asarray([(arr[:,i]-means)/(std+0.0001) for i in range(len(arr[0]))])
new = np.swapaxes(res, 0,1)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

data = compute_class_weight("auto", np.unique(cf["Pitch Type"].values),cf["Pitch Type"].values )

In [ ]:
labels_string= cf["Pitch Type"].values
unique =np.unique(labels_string)
#print(np.unique(pitches))
#print(np.sum(data))
nr_classes=len(unique)
ex_per_class = 4

"""liste=np.zeros((7,4))
for i, types in enumerate(np.unique(pitches)):
    liste[i] = (np.random.choice(np.where(pitches==types)[0], 4))
print(liste)
"""
index_liste = []
for pitches in unique:
    index_liste.append(np.where(labels_string==pitches))
    
def balanced_batches(y):   
    for j in range(5):
        liste=np.zeros((nr_classes, ex_per_class))
        for i in range(nr_classes):
            liste[i] = np.random.choice(index_liste[i][0], ex_per_class)
        #print(liste.flatten().astype(int))
        yield y[liste.flatten().astype(int)]

In [ ]:
for batch_x in balanced_batches(labels_string):
    print(batch_x)

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
#from sklearn.preprocessing import StandardScaler
from data_preprocess import Preprocessor

def decode_one_hot(results, unique):
    """takes the maximum value and gets the corresponding pitch type
    input: array of size trials * pitchTypesNr
    returns: array of size trials containing the pitch type as a string
    """
    #unique  = np.unique(cf["Pitch Type"].values)
    p = []
    for _, pitch in enumerate(results):
        ind = np.argmax(pitch)
        if pitch[ind]>0.5:
            p.append(unique[ind])
        else:
            p.append("Too small")
    return p

leaky_relu = lambda x: tf.maximum(0.2*x, x)


ex_per_class = 4
EPOCHS = 10
PATH = "/Users/ninawiedemann/Desktop/UNI/Praktikum/sv_data.csv"
LABELS = "Pitch Type"
act = leaky_relu
CUT_OFF_Classes = 60

prepro = Preprocessor(PATH, CUT_OFF_Classes)
data = np.load("coord_sv.npy") #prepro.get_coord_arr("coord_sv.npy")

M,N,nr_joints,_ = data.shape
SEP = int(M*0.9)

labels, unique = prepro.get_labels_onehot(LABELS)
labels_string = prepro.get_labels(LABELS)
#labels_test = decode_one_hot(labels[SEP:, :], unique)

nr_classes = len(np.unique(labels_string))
BATCHSIZE = nr_classes*ex_per_class
print("nr classes", nr_classes, "Batchsize", BATCHSIZE)

# NET

ind = np.random.permutation(len(data))
train_ind = ind[:SEP]
test_ind = ind[SEP:]

train_x = data[train_ind]
test_x = data[test_ind]
train_t= labels[train_ind]
test_t = labels[test_ind]
labels_string_train = labels_string[train_ind]
labels_string_test = labels_string[test_ind]

"""
DATA TESTING:
indiuh = np.where(ind==2000)
print("Labels nach preprocc von 2000", labels_string[2000])
print("new Index of 2000", indiuh, "test ob where funkt: ", ind[indiuh])
print("train coord of 2000 u 140", train_x[indiuh, 140])
print("labels_string von 2000", labels_string_train[indiuh])
print("one hot von 2000", train_t[indiuh])
"""

index_liste = []
for pitches in unique:
    index_liste.append(np.where(labels_string_train==pitches))

len_test = len(test_x)
len_train = len(train_x)
print("Test set size: ", len_test, " train set size: ", len_train)
print("Shapes of train_x", train_x.shape, "shape of test_x", test_x.shape)

# normal conv net

In [ ]:
x = tf.placeholder(tf.float32, (None, N, nr_joints, 2), name = "input")
x_ = tf.reshape(x, (-1, N, nr_joints*2))
y = tf.placeholder(tf.float32, (None, len(labels[0])))

net = tf.layers.conv1d(x_, filters=256, kernel_size=5, strides=2, activation=act)
net = tf.layers.conv1d(net, filters=256, kernel_size=3, strides=1, activation=act)
net = tf.layers.conv1d(net, filters=128, kernel_size=3, strides=1, activation=act)
net = tf.layers.conv1d(net, filters=1, kernel_size=1, activation = act)
shapes = net.get_shape().as_list()
ff = tf.reshape(net, (-1, shapes[1]*shapes[2]))
ff = tf.layers.dense(ff, 1024, activation = act)
ff = tf.layers.dense(ff, 128, activation = act)
logits = tf.layers.dense(ff, len(labels[0]), activation = None)
out = tf.nn.softmax(logits)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits)) # tf.reduce_mean(tf.square(y-ff))
optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)

# basic lstm

In [ ]:
import tensorflow as tf
from tensorflow.contrib import rnn

learning_rate = 0.001
training_iters = 100000
batch_size = BATCHSIZE
display_step = 10

tf.reset_default_graph()

# Network Parameters
n_input = nr_joints*2 # MNIST data input (img shape: 28*28)
n_steps = N # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 12 # MNIST total classes (0-9 digits)

# tf Graph input
x_ = tf.placeholder("float", [None, n_steps, nr_joints, 2])
x = tf.reshape(x_, (-1, N, nr_joints*2))
y = tf.placeholder("float", [None, n_classes])

"""# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}"""


def RNN(x):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, n_steps, 1)
    
    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    """with tf.variable_scope("myrnn") as scope:
        for i in range(n_steps-1):
            if i > 0:
                scope.reuse_variables()
            output, state = lstm_cell(x[i], state)"""

    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.layers.dense(outputs[-1], n_classes)   #tf.matmul(outputs[-1], weights['out']) + biases['out']

out_logits = RNN(x)
out = tf.nn.softmax(out_logits)

# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out_logits, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# multi layer lstm

In [ ]:
import tensorflow as tf
from tensorflow.contrib import rnn

learning_rate = 0.001
training_iters = 100000
batch_size = BATCHSIZE
display_step = 10
nr_layers = 4

tf.reset_default_graph()

# Network Parameters
n_input = nr_joints*2 # MNIST data input (img shape: 28*28)
n_steps = N # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 12 # MNIST total classes (0-9 digits)

# tf Graph input
x_ = tf.placeholder("float", [None, n_steps, nr_joints, 2])
x = tf.reshape(x_, (-1, N, nr_joints*2))
y = tf.placeholder("float", [None, n_classes])

"""# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}"""


def RNN(x):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, n_steps, 1)
    
    # Define a lstm cell with tensorflow
    #lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    
    def lstm_cell():
          return rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    
    stacked_lstm = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(nr_layers)])


    # Get lstm cell output
    """with tf.variable_scope("myrnn") as scope:
        for i in range(n_steps-1):
            if i > 0:
                scope.reuse_variables()
            output, state = lstm_cell(x[i], state)"""

    outputs, states = rnn.static_rnn(stacked_lstm, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.layers.dense(outputs[-1], n_classes)   #tf.matmul(outputs[-1], weights['out']) + biases['out']

out_logits = RNN(x)
out = tf.nn.softmax(out_logits)

# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out_logits, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# training

In [ ]:
def batches(x, y, batchsize=32):
    permute = np.random.permutation(len(x))
    for i in range(0, len(x)-batchsize, batchsize):
        indices = permute[i:i+batchsize]
        yield x[indices], y[indices]

def balanced_batches(x, y, batchsize=32):
    for j in range(200):
        liste=np.zeros((nr_classes, ex_per_class))
        for i in range(nr_classes):
            # print(j, i, np.random.choice(index_liste[i][0], ex_per_class))
            liste[i] = np.random.choice(index_liste[i][0], ex_per_class, replace=False)
        liste = liste.flatten().astype(int)
        yield x[liste], y[liste]

sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Run session for EPOCH epochs
for epoch in range(EPOCHS + 1):
    for batch_x, batch_t in balanced_batches(train_x, train_t, BATCHSIZE):
        sess.run(optimizer, {x_: batch_x, y: batch_t})
    # print("Loss test: ", sess.run(loss, {x: test_x, y: test_t}))
    #print("Loss train: ", sess.run(loss, {x: train_x, y: train_t}))

    #Test Accuracy
    loss_test, out_test = sess.run([loss,out], {x_: test_x, y: test_t})
    print("Loss test", loss_test)
    pitches_test = decode_one_hot(out_test, unique)
    print("Accuracy test: ", np.sum(np.asarray(labels_string_test)==pitches_test)/len_test)
    
    #Train Accuracy
    out_train = sess.run(out, {x_: train_x, y: train_t})
    pitches_train = decode_one_hot(out_train, unique)
    print("Accuracy train: ", np.sum(np.asarray(labels_string_train)==pitches_train)/SEP)

print(pitches_test)

# cut file to stretch and pitch type

In [ ]:
import numpy as np

out_list = ["ine", "ine", "bdeu", "hdba", "nina", "nina"]
ground_truth_list = ["ine","ine", "bdeu", "ine", "nina", "halo"]

out = np.array(out_list)
ground_truth = np.array(ground_truth_list)

same = out[np.where(out==ground_truth)[0]]

right_frequency = sp.stats.itemfreq(same)
total_frequency = sp.stats.itemfreq(ground_truth)
right_dict = dict(zip(right_frequency[:,0], right_frequency[:,1]))
total_dict = dict(zip(total_frequency[:,0], total_frequency[:,1]))

acc= right_dict
for types in right_dict.keys():
    acc[types] = (int(right_dict[types])/float(total_dict[types]))


print(acc)

In [ ]:
#cf = np.load("coord_array.npy")
#sv = np.load("coord_sv.npy")
#print(cf.shape, sv.shape)

cf = pd.read_csv("cf_data.csv")
sv = pd.read_csv("sv_data.csv")


In [ ]:
cf = cf[cf["Player"]=="Pitcher"]
sv = sv[sv["Player"]=="Pitcher"]
cf_plays = cf['play_id'].values
print(sv[sv["play_id"]==cf_plays[0]].index)

In [ ]:
print(cf_plays)

In [ ]:
print(sv["play_id"].values)

In [ ]:
common_ids = []
sv_plays = sv["play_id"].values
for i, ids in enumerate(cf_plays):
    if ids in sv_plays:
        common_ids.append(ids)
        

In [ ]:
print(len(common_ids))
print(len(sv_plays))
print(len(cf_plays))

In [ ]:
print(cf["play_id"].values[0])
new = cf[cf["play_id"] in sv["play_id"].values]
print((np.array(sv_plays)==np.array(cf_plays)))
print(np.where(np.array(sv_plays)==np.array(cf_plays)))

In [ ]:
cf = pd.read_csv("cf_data.csv")
sv = pd.read_csv("sv_data.csv")


cf = cf[cf["Player"]=="Pitcher"]
sv = sv[sv["Player"]=="Pitcher"]


In [ ]:
pitcher = cf["Pitcher"].values.astype(int)
statistic = sp.stats.itemfreq(pitcher) #.sort(axis = 0)

number = np.array(frequ[:,1])

for i in range(5):
    maxi = np.argmax(number)
    a = frequ[maxi]
    number[maxi]=0
    print(a)

In [ ]:
split = cf[cf["Runner on 1st"]!=None]
print(split)

In [9]:
import pandas as pd
import numpy as np

from data_preprocess import Preprocessor

#pre_cf = Preprocessor("cf_data.csv",60)
#new_data = pre_cf.concat_with_second("sv_data.csv")
#print(new_data)

PATH = "cf_data.csv"
LABELS = "Pitch Type"

prepro = Preprocessor(PATH)
players = prepro.get_list_with_most("Pitcher")
print(players)
#

/Users/ninawiedemann/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (253,254,255,256,257,258,259,289) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


csv eingelesen with length  13150
Only Pitcher rows
([527054.0, 285079.0, 592314.0, 448802.0, 656794.0], array([[  1.12526000e+05,   2.40000000e+02],
       [  2.76351000e+05,   4.00000000e+00],
       [  2.85079000e+05,   4.35000000e+02],
       [  4.07793000e+05,   4.40000000e+01],
       [  4.07822000e+05,   1.00000000e+01],
       [  4.24144000e+05,   1.00000000e+01],
       [  4.30661000e+05,   8.00000000e+00],
       [  4.30912000e+05,   4.00000000e+01],
       [  4.33217000e+05,   3.00000000e+00],
       [  4.34442000e+05,   1.70000000e+01],
       [  4.35400000e+05,   1.42000000e+02],
       [  4.44468000e+05,   1.10000000e+01],
       [  4.46399000e+05,   4.00000000e+00],
       [  4.46899000e+05,   1.00000000e+01],
       [  4.47714000e+05,   7.70000000e+01],
       [  4.48609000e+05,   8.00000000e+00],
       [  4.48614000e+05,   8.00000000e+00],
       [  4.48802000e+05,   3.25000000e+02],
       [  4.50212000e+05,   2.00000000e+00],
       [  4.50308000e+05,   2.60000000e+

In [10]:
prepro.cut_file_to_pitcher(527054.0)
prepro.get_list_with_most("Pitch Type")


(['Fastball (4-seam)', 'Slider', 'Fastball (2-seam)', 'Curveball', 'Changeup'],
 array([['Changeup', 29],
        ['Curveball', 39],
        ['Fastball (2-seam)', 82],
        ['Fastball (4-seam)', 226],
        ['Slider', 87]], dtype=object))

In [ ]:
print(cf.columns.tolist())

# histogram of 5 players with most pitches

In [ ]:
from data_preprocess import Preprocessor
prepro = Preprocessor("cf_data.csv")
players, _ = prepro.get_list_with_most("Pitcher")
#print(players[2])
pitchi = []
for i in range(5):
    prepro = Preprocessor("cf_data.csv")
    prepro.cut_file_to_pitcher(players[i])
    _, stat = prepro.get_list_with_most("Pitch Type")
    pitchi.append(stat)
print(pitchi)

In [ ]:
for i in range(5):
    print(players[i], ": ", np.sum(pitchi[i][:, 1]))

In [ ]:
#data=np.random.random((4,10))
import matplotlib.pylab as plt
import numpy as np

titles = players
for i in range(5):
    
    plt.figure(figsize=(20,10))
    plt.ylim([0,350])
    y_pos = range(len(pitchi[i]))
    plt.bar(y_pos, pitchi[i][:,1], align='center', alpha=0.5, width = 1)
    #plt.ylim([0,350])
    plt.xticks(y_pos, pitchi[i][:,0])
    plt.ylabel('number of videos')
    plt.title(titles[i])
    plt.show()

In [ ]:
import shutil
shutil.rmtree("logs")

# Try to plot coordinates

In [ ]:
import pandas as pd
import numpy as np

from data_preprocess import Preprocessor
prepro = Preprocessor("cf_data.csv")

In [ ]:
players, _ = prepro.get_list_with_most("Pitcher")
prepro.select_movement("Stretch")
prepro.cut_file_to_pitcher(players[2]) 

In [ ]:
df = prepro.cf
pitchi, _ = prepro.get_list_with_most("Pitch Type")
df = df[df["Pitch Type"]==pitchi[0]]

In [ ]:
prepro.cf = df

In [ ]:
data = prepro.get_coord_arr()

In [ ]:
print(data.shape)
print(data)

In [ ]:
#print(data[10])

In [ ]:
print(data.shape)
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(16,10))
ax = fig.add_subplot(111, projection='3d')

#z=range(167)
for i in range(167):
    ax.scatter(data[10, i, :16, 0], data[10, i, :16, 1], i, c= 'red', s=4)


#for i in range(18):
#    ax.scatter(data[10, :, i, 0], data[10, i, :, 1], z, c= 'red', s=1)
    
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()
#Axes3D.scatter(mean_data[0], mean_data[1], mean_data[2])

In [ ]:
print(max(prepro.cf['first_movement_frame_index'].values))


In [ ]:
import pandas as pd
cf = pd.read_csv("cf_data.csv")

# Ergebnisse

Best model for one player (here: the one with most trials) and all his pitch types (Stretch&windup mixed):

Convolutional NN with 4 conv layers and 3 fully connected feed forward layers, using regularization (L2 loss for weights and Dropout)

Frames aligned by first_movement_frame, head coordinates removed (only first 12)

nr classes 5 Batchsize 20 trained for 40 epochs, 100 balanced batches per epoch

classes:  ['Changeup', 'Curveball', 'Fastball (2-seam)', 'Fastball (4-seam)', 'Slider']

data shape: (463, 92, 12, 2) label_shape (463, 5) (463,)

Test set size:  47  train set size:  416

Shapes of train_x (416, 92, 12, 2) shape of test_x (47, 92, 12, 2)

# shorter version of conv net

all with alignment and concatenated data

conv_short version,      mit max_pooling, with conv-batch_norm-dropout architecture
    
player0: 53% (5 classes), 46,                  35

player1: 80% (2 classes), 85,                  70

player2: 27% (6 classes), 21,                  35

player3: 45% (5 classes), 23,                  36

player4: 35% (4 classes), 52,                  46

Same but only one pitchtype:

and concat data and alignment with conv-batch_norm-dropout architecture

player0: 62

player1: 77

player2: 75

player3: 59

player4: 84

In [ ]:
"""
Trained on concatenated data from cf and sv (4 coordinates) on only Windup 53%, only Stretch 54%, all 50%
(without alignment and without regularization loss)
"""
def best_in_cluster_concat53(self, x, nr_classes, training, rate_dropout=0.6, act=tf.nn.relu):
        shape = x.get_shape().as_list()
        x_ = tf.reshape(x, (-1, shape[1], shape[2]*shape[3]))
        net = tf.layers.conv1d(x_, filters=256, kernel_size=5, strides=2, activation=act, name="conv1")
        tf.summary.histogram("conv_1_layer", net)
        # net = tf.layers.dropout(net, rate=rate_dropout, training=training)
        net = tf.layers.conv1d(net, filters=128, kernel_size=3, activation = act, name="conv4")
        shapes = net.get_shape().as_list()
        ff = tf.reshape(net, (-1, shapes[1]*shapes[2]))
        logits = tf.layers.dense(ff, nr_classes, activation = None, name = "ff3")
        out = tf.nn.softmax(logits)
        return out, logits
    
"""Trained on only one pitch type: about 75% with higher batchsize 78
"""

# Align pitches by "first_movement_frame_index"

In [ ]:
release_frame = prepro.cf["first_movement_frame_index"].values

def align_frames(data, release_frame, fr_before, fr_after):
    """
    Takes the data and an array that indicates the frame number of the first movement
    cuts all data to fr_before the release_frame and fr_after the release frame
    returns an array of size data.shape except the second dimension is cut to length fr_before+fr_after
    """
    M, _, nr_joints, nr_coord = data.shape
    new = np.zeros((M, fr_after+fr_before, nr_joints, nr_coord))
    for i, row in enumerate(data):
        ind = release_frame[i]
        if pd.isnull(ind):
            ind = np.mean(release_frame[:i])
        start = int(ind-fr_before)
        end = int(ind+ fr_after)
        #print(start, end)
        new[i] = data[i, start:end, :,:]
    return new

# RNN/ LSTM
tf learn module to create a RNN 

In [ ]:
import tflearn
def get_network_wide(frames, input_size, num_classes):
    """Create a one-layer LSTM"""
    net = tflearn.input_data(shape=[None, frames, input_size])
    net = tflearn.lstm(net, 256, dropout=0.2)
    net = tflearn.fully_connected(net, num_classes, activation='softmax')
    net = tflearn.regression(net, optimizer='adam',
                             loss='categorical_crossentropy', name='output1')
    return net
model = DNN(get_network_wide(N, nr_joints*nr_coordinates, nr_classes))
model.fit(X, Y)

In [ ]:
print("first_movement_frame_index")
print(prepro.cf["first_movement_frame_index"].values[:50])

print("'pitch_frame_index'")
print(np.where(prepro.cf['pitch_frame_index'].values<0))

# Training on best script so far (54% on all data)

regularization best weighted 0.0005

dropout made it worse

tan also up to 50, leaky relu same

aligned made it worse

only one conv layer: also 54% but overfitting

TRAIN IT ON JUST HEAD COORDINATES: (13:18) 40%, ON JUST ONE COORDINATE(14): 30% --> something wrong

In [ ]:
ex = pd.read_csv("cf_data.csv", converters = {'0': eval, '1': eval, '2': eval, '3': eval, '4': eval, '5': eval, '6': eval, '7': eval, '8': eval, '9': eval, '10': eval, '11': eval, '12': eval, '13': eval, '14': eval, '15': eval, '16': eval, '17': eval, '18': eval, '19': eval, '20': eval, '21': eval, '22': eval, '23': eval, '24': eval, '25': eval, '26': eval, '27': eval, '28': eval, '29': eval, '30': eval, '31': eval, '32': eval, '33': eval, '34': eval, '35': eval, '36': eval, '37': eval, '38': eval, '39': eval, '40': eval, '41': eval, '42': eval, '43': eval, '44': eval, '45': eval, '46': eval, '47': eval, '48': eval, '49': eval, '50': eval, '51': eval, '52': eval, '53': eval, '54': eval, '55': eval, '56': eval, '57': eval, '58': eval, '59': eval, '60': eval, '61': eval, '62': eval, '63': eval, '64': eval, '65': eval, '66': eval, '67': eval, '68': eval, '69': eval, '70': eval, '71': eval, '72': eval, '73': eval, '74': eval, '75': eval, '76': eval, '77': eval, '78': eval, '79': eval, '80': eval, '81': eval, '82': eval, '83': eval, '84': eval, '85': eval, '86': eval, '87': eval, '88': eval, '89': eval, '90': eval, '91': eval, '92': eval, '93': eval, '94': eval, '95': eval, '96': eval, '97': eval, '98': eval, '99': eval, '100': eval, '101': eval, '102': eval, '103': eval, '104': eval, '105': eval, '106': eval, '107': eval, '108': eval, '109': eval, '110': eval, '111': eval, '112': eval, '113': eval, '114': eval, '115': eval, '116': eval, '117': eval, '118': eval, '119': eval, '120': eval, '121': eval, '122': eval, '123': eval, '124': eval, '125': eval, '126': eval, '127': eval, '128': eval, '129': eval, '130': eval, '131': eval, '132': eval, '133': eval, '134': eval, '135': eval, '136': eval, '137': eval, '138': eval, '139': eval, '140': eval, '141': eval, '142': eval, '143': eval, '144': eval, '145': eval, '146': eval, '147': eval, '148': eval, '149': eval, '150': eval, '151': eval, '152': eval, '153': eval, '154': eval, '155': eval, '156': eval, '157': eval, '158': eval, '159': eval, '160': eval, '161': eval, '162': eval, '163': eval, '164': eval, '165': eval, '166': eval})

In [ ]:
import sys
for i in range(167):
    sys.stdout.write("'"+str(i)+"': eval, ")

In [ ]:
df ={"Batchsize": 40., "Epochs": 50., "batch_nr_in_epoch": 100., "file": "cf", "act": tf.nn.relu,
                   "CUT_OFF_Classes": 50., "dropout": 0.6, "learning_rate": 0.0001, "lstm_units": 4., 
                   "lstm_hidden_layers": 128.}

In [ ]:
df =pd.DataFrame({"Batchsize": pd.Series(40.), "Epochs": pd.Series(50.), "batch_nr_in_epoch": pd.Series(100.), 
                  "file": pd.Series("cf"), "act": pd.Series(tf.nn.relu), "nr_joints": pd.Series(12),
                   "CUT_OFF_Classes": pd.Series(50.), "dropout": pd.Series(0.6), "learning_rate": pd.Series(0.0001),
                  "lstm_units": pd.Series(4.),"lstm_hidden_layers": pd.Series(128.), "loss": pd.Series([[5,4,3,2,1]]),
                 "test_acc": pd.Series([[3,4,5,6]]), "highest_acc": pd.Series(10), "nr_classes": pd.Series(5), "align": pd.Series(True)
                 , "train_acc": pd.Series([[3,4,5]]), "len_train": pd.Series(2000), "model":pd.Series("conv2d")})
df.to_csv("test_parameters.csv")
#{"Batchsize": 40., "Epochs": 50.}
#df["Batchsize"]=pd.Series(40)
#df["Epoch"]=10
#a = pd.DataFrame.from_dict(df, orient = "index")

In [ ]:
new = pd.read_csv("test_parameters.csv")
columns = new.columns.values.tolist()
print(len(columns))
#add = pd.DataFrame([[0, 20,30 , 12, 60, tf.nn.tanh,[1,2,3], 0.4, "sv", 0.01, 256, 5]], columns=columns)
#print(add)
#concat = new.append(add, ignore_index = True)
#print(new)
#print(concat["coordinates_used"])
#cut = new.drop(new.columns[[0]], axis=1, inplace=True)
#print(cut.columns.values.tolist())

In [ ]:
new = pd.read_csv("test_parameters.csv")
columns = new.columns.values.tolist()
add = pd.DataFrame([[0, BATCHSIZE, CUT_OFF_Classes , EPOCHS, act, align, batch_nr_in_epoch, rate_dropout
                     , PATH, max(acc_test), learning_rate, loss, n_hidden, nr_layers, nr_classes, nr_joints, test_acc, train_acc]], columns=columns)
concat = new.append(add, ignore_index = True)
concat.to_csv("new")

Drop one row of csv file

In [ ]:
inde = pd.read_csv("test_parameters.csv")
a = inde.drop(inde.columns[[0]], axis=1, inplace=True)
#print(a["Epochs"])
a.to_csv("test_parameters.csv")

In [ ]:
EPOCHS = 80
batch_nr_in_epoch = 100
align = False
act = tf.nn.relu
rate_dropout = 0
learning_rate = 0.0005
nr_layers = 4
n_hidden = 128
BATCHSIZE = 40
network = "conv1d (256, 5) - conv1d(128, 3) - dense(nr_classes) - softmax"
CUT_OFF_Classes = 50
PATH = "sv"
len_train = 5605
losses= [6,5,4]
nr_classes = 9
nr_joints =12
acc_train = [1,2,3]
acc_test = [2,3,6]

In [ ]:
new = pd.read_csv("test_parameters_testing.csv")
columns = new.columns.values.tolist()# .append("optimizer")
    # print(len(columns))
    # print("Written to csv: ", BATCHSIZE, CUT_OFF_Classes , EPOCHS, act, align, batch_nr_in_epoch, rate_dropout
    #                      , PATH, max(acc_test), learning_rate, len_train, losses, n_hidden, nr_layers, network, nr_classes,
    #                      nr_joints, acc_train, acc_test)

add = pd.DataFrame([[0, BATCHSIZE, CUT_OFF_Classes , EPOCHS, act, align, batch_nr_in_epoch, rate_dropout
                     , PATH, max(acc_test), learning_rate, len_train, losses, n_hidden, nr_layers, network, nr_classes,
                     nr_joints, acc_train, acc_test, "sgd", "dhe"]], columns=columns)
concat = new.append(add, ignore_index = True)
concat["optimizer"]= "sgd"
concat.drop(concat.columns[[0]], axis=1, inplace=True)
concat.to_csv("test_parameters_testing.csv")

In [ ]:
df = pd.read_csv("test_parameters.csv")
columns = df.columns.values.tolist()
print(eval(columns))
# df = df.drop(df.index[[0,1,2,3,4,5]])
# df.drop(df.columns[[0,1,2,3]], axis=1, inplace=True)
#df["1st_hidden_dense"]= pd.Series(1024)
#df["1st_hidden_dense"]= 1024
#df["2nd_hidden_dense"]= pd.Series(128)
#df["2nd_hidden_dense"]= 128
#df.drop(df.columns[[0]], axis=1, inplace=True)
#df.to_csv("test_parameters.csv")

In [ ]:
csv = pd.read_csv("beispiel.csv")
cf = pd.read_csv("cf_data.csv")

In [ ]:
cf = pd.read_csv("cf_data.csv")

In [ ]:
import scipy as sp
import scipy.stats
test = cf["Game"].values
#print(np.unique(test))
print(max(sp.stats.itemfreq(test)[:, 1]))
#print(cf.iloc[3086], cf.iloc[9464])
#print(len(cf))
#print(len(cf.iloc[3086].values))
eins = cf.iloc[3086].values
zwei = cf.iloc[9464].values
print(test[3086], test[9464])
for i in range(len(eins)):
    if eins[i]!=zwei[i]:
        print(cf.columns.tolist()[i], eins[i], zwei[i])

In [ ]:
from os import listdir

directory = "/Users/ninawiedemann/Desktop/UNI/Praktikum/output/"
list_files = listdir(directory)
start_frames = cf.columns.get_loc("0")
#print(list_files)
for file in list_files:
    if file[0]!='.':
        csv = pd.read_csv(directory+file)
        #print(csv["Unnamed: 0"].values)
        game_id = csv[csv["Frame"]=="Game"]["Pitcher_player"].values[0]
        last_frame = eval(csv[csv["Frame"]=="# of last frame"]["Pitcher_player"].values[0])+1
        print(game_id, last_frame)
        #ind = cf[cf["play_id"]==game_id].index.values
        #print(ind)

        #print(cf["play_id"].values)
        #print(cf.columns.tolist())
        
        location_play = cf[cf["Game"]==game_id].index.values
        print(location_play)
        if len(location_play)==4:
            cf.drop(cf.index[[location_play[2], location_play[3]]])
        # get index of play_id in cf
        for i in range(last_frame):
            pitcher_arr = csv["Pitcher_player"].values[i].replace('\n', ',').replace('[  ', '[').replace("  ", ",").replace(",,",",")
            batter_arr = csv["Batter_player"].values[i].replace('\n', ',').replace('[  ', '[').replace("  ", ",").replace(",,",",")
            
            cf.iloc[int(location_play[0]), start_frames+i] = pitcher_arr #csv["Pitcher_player"].values[i]
            cf.iloc[int(location_play[1]), start_frames+i] = batter_arr #csv["Batter_player"].values[i]
            # check if right order of pitcher and batter

cf.to_csv("unprocessed_data.csv")

print(len(cf.values))

# Interpolated data

In [ ]:
unpro = pd.read_csv("unprocessed_data.csv")

In [ ]:
lines = unpro[unpro["Game"]=="490780-7be4d62a-6ddb-41e2-9331-dd9656df1819"] #.index.values
#print(lines)
bsp  = lines["162"].values[1]
# print(repr(bsp))
#besp = bsp.replace('\n', ',').replace('[  ', '[').replace("  ", ",").replace(",,",",")
#print(besp)
#import ast
print(eval(bsp))

In [ ]:
df = pd.read_csv("cf_data.csv")

In [ ]:
lines = df[df["Game"]=="490780-7be4d62a-6ddb-41e2-9331-dd9656df1819"]
bsp1 = lines["162"].values[1]
print(bsp1)
print(eval(bsp1))

In [ ]:
test = unpro["Game"].values
#print(np.unique(test))
print(max(sp.stats.itemfreq(test)[:, 1]))

In [ ]:
from data_preprocess import Preprocessor
pre = Preprocessor("unprocessed_data.csv")
pre.remove_small_classes(50)

In [14]:
data = np.load("unpro_all_coord.npy")
print(np.amax(data), np.amin(data))

303.0 0.0


In [ ]:
M,N, nr_j, _ = data.shape
#flat = np.flatten()
#print(np.where(data[:,:,:, 0].flatten()==0 and data[:,:,:, 1]!=0))
print(data.shape)
data = data[:,:,:12,:]

for i in range(M):
    for j in range(N):
        for k in range(12):
            if k!=16 and data[i,j,k,0]==0:
                if j<150:
                    print(i,j,k)
                ind  = np.where(data[i, :, k, 0]>0)[0]
                if ind[-1]>j and ind[0]<j: #ind!=np.array([]) and 
                    #print(ind)
                    ind_before = ind[ind<j][-1]
                    ind_after = ind[ind>j][0]
                    inter0 = np.linspace(data[i, ind_before, k, 0], data[i, ind_after, k, 0], ind_after-ind_before, endpoint = False)
                    inter1 = np.linspace(data[i, ind_before, k, 1], data[i, ind_after, k, 1], ind_after-ind_before, endpoint = False)
                    #print(ind)
                    #print(i,j,k)
                    #print(ind)
                    #print(data[i,j])
                    #print(data[i, j:ind_after, k, 0])
                    data[i,j:ind_after,k,0]= inter0[1:]
                    data[i,j:ind_after,k,1]= inter1[1:]
                    #print(data[i,j])
                elif ind[-1]>j:
                    ind_before = ind[ind>j][0]  
                    data[i,j,k,0] = data[i,ind_before,k,0]
                    data[i,j,k,1] = data[i,ind_before,k,1]
                elif ind[0]<j:
                    ind_after = ind[ind<j][-1]
                    data[i,j,k,0] = data[i,ind_after,k,0]
                    data[i,j,k,1] = data[i,ind_after,k,1]
                    
print(data.shape)
print(data)
np.save("interpolated.npy", data)

In [ ]:
fitness = [(524, ['relu', 0.0, True, 0.001, 0.0050000000000000001, 1024, 5, 512, 7, 0, 256]), (658, ['relu', 0.59999999999999998, False, 0.00050000000000000001, 0.00050000000000000001, 512, 7, 512, 11, 128, 0]), (146, ['leaky', 0.59999999999999998, False, 0.00050000000000000001, 0.0, 128, 11, 0, 7, 128, 256]), (390, ['leaky', 0.59999999999999998, False, 0.00050000000000000001, 0.0, 128, 3, 256, 3, 128, 0]), (12, ['relu', 0.59999999999999998, False, 0.00050000000000000001, 0.0050000000000000001, 256, 3, 0, 9, 0, 256]), (270, ['leaky', 0.59999999999999998, True, 0.00025000000000000001, 0.0, 0, 5, 128, 9, 128, 256]), (404, ['leaky', 0.0, False, 0.00050000000000000001, 0.00050000000000000001, 128, 11, 128, 9, 256, 128]), (1170, ['relu', 0.0, True, 0.00050000000000000001, 0.00050000000000000001, 512, 9, 128, 9, 1024, 0]), (650, ['leaky', 0.0, False, 0.0001, 0.0, 512, 5, 512, 5, 128, 128]), (1036, ['relu', 0.0, False, 0.001, 0.001, 128, 7, 0, 5, 1024, 1024])]

# Missing value by ML

## Try on prednet (https://github.com/coxlab/prednet) but code did not work

### Data preprocessing for that:

In [ ]:
data = np.load("interpolated.npy")
print(data.shape)

source = np.zeros((6422*167))
for i in range(6422):
    source[i*167:(i+1)*167] = i

data_images = np.reshape(data, (6422*167, 12,2))

split1 = 6000*167
split2 = 6400*167
data_train = data_images[:split1]
data_val = data_images[split1:split2]
data_test = data_images[split2:]
sources_train = source[:split1]
sources_val = source[split1:split2]
sources_test = source[split2:]

print(data_train.shape, data_val.shape, data_test.shape, sources_train.shape, sources_val.shape, sources_test.shape)

np.save("X_train.npy", data_train)
np.save("X_val.npy", data_val)
np.save("X_test.npy", data_test)
np.save("sources_train.npy", sources_train)
np.save("sources_val.npy", sources_val)
np.save("sources_test.npy", sources_test)

train_file = np.expand_dims(np.load("DATA_DIR/X_train.npy"), axis = 3)
train_sources = np.expand_dims(np.load("DATA_DIR/sources_train.npy"), axis = 1) #os.path.join(DATA_DIR, 'sources_train.hkl
val_file = np.expand_dims(np.load("DATA_DIR/X_val.npy"), axis = 3) #os.path.join(DATA_DIR, 'X_val.hkl')
val_sources = np.expand_dims(np.load("DATA_DIR/sources_val.npy"), axis = 1)